# Multi-Agent Orchestration Demo

This notebook demonstrates a minimal v0 orchestration across `planner -> retrieval -> analyst`.

Flow:
1. Planner performs metadata extraction, intent classification, and query expansion.
2. If `retrieval_needed=True`, retrieval agent is invoked with planner outputs.
3. Retrieval output is converted to `AnalystPacket` and sent to analyst agent.
4. If retrieval is skipped or blocked by missing metadata, analyst still runs with planner context and open issues.

Note: This is intentionally minimal for demonstration; quality-based iterative replanning can be added next.


In [1]:
from __future__ import annotations
import sys
sys.path.insert(0, '../src')

import json

from agents.orchestrator import run_multi_agent_orchestration



In [2]:
# Orchestration implementation lives in src/agents/orchestrator/agent_orchestrator.py
# This notebook acts as a thin demo harness.



In [3]:
# Example query
user_query = "What was Apple’s total debt (short-term plus long-term) at year-end 2024?"

result = await run_multi_agent_orchestration(user_query, debug=True)

summary = {
    'run_id': result['run_id'],
    'intent': result['planner']['intent'],
    'retrieval_needed': result['planner']['retrieval_needed'],
    'queries': result['planner']['query_bundle']['queries'],
    'retrieval_ok': (result['retrieval'] or {}).get('ok') if isinstance(result['retrieval'], dict) else None,
    'retrieved_tables': len((result['retrieval'] or {}).get('top_tables', [])) if isinstance(result['retrieval'], dict) else 0,
    'analyst_used_financial_evaluator': result['analyst']['trace']['used_financial_evaluator'],
    'answer': result['analyst']['answer'],
    'open_issues': result['analyst']['open_issues'],
    'trace': result['orchestrator_trace'],
}

print(json.dumps(summary, indent=2, ensure_ascii=False))


[planner_timing_ms] llm_planner_ms=58863 llm_query_expansion_ms=7425 plan_total_ms=66298 step_expand_queries_ms=7425 step_fallback_ms=0 step_llm_plan_ms=58864 step_preextract_ms=0 step_validate_ms=0
[analyst_timing_ms] build_prompt_ms=0 agent_invoke_ms=103116 agent_retry_ms=0 total_ms=103116
[analyst_debug] tool_calls=1 used_financial_evaluator=True
{
  "run_id": "run-053d0b41",
  "intent": "filing_calc",
  "retrieval_needed": true,
  "queries": [
    "What was Apple’s total debt (short-term plus long-term) at year-end 2024?",
    "Total liabilities",
    "Total term debt",
    "Commercial paper",
    "Less: Current portion of term debt"
  ],
  "retrieval_ok": true,
  "retrieved_tables": 1,
  "analyst_used_financial_evaluator": true,
  "answer": "Apple’s total debt (short-term plus long-term) at year-end 2024 was **$96,662 million**.",
  "open_issues": [
    {
      "code": "COMPANY_NAME_DETECTED",
      "message": "Detected company name 'Apple'.",
      "severity": "info"
    },
    {

In [4]:
# Optional: inspect full payload
print(json.dumps(result, indent=2, ensure_ascii=False))


{
  "run_id": "run-053d0b41",
  "planner": {
    "retrieval_needed": true,
    "intent": "filing_calc",
    "metadata": {
      "ticker": "AAPL",
      "company_name": "Apple",
      "fiscal_year": 2024,
      "form_type": "10-K",
      "doc_types": [
        "table"
      ],
      "fiscal_quarter": null,
      "section_hints": [
        "Liquidity and Capital Resources",
        "Debt",
        "Notes to Consolidated Financial Statements"
      ],
      "units_hint": []
    },
    "query_bundle": {
      "base_query": "What was Apple’s total debt (short-term plus long-term) at year-end 2024?",
      "queries": [
        "What was Apple’s total debt (short-term plus long-term) at year-end 2024?",
        "Total liabilities",
        "Total term debt",
        "Commercial paper",
        "Less: Current portion of term debt"
      ],
      "must_include": [
        "debt",
        "borrow",
        "credit",
        "notes payable",
        "long-term",
        "current portion"
      ],

In [2]:
# E2E smoke test (post-reorg)
test_query = "What was Apple’s total debt (short-term plus long-term) at year-end 2024?"

result = await run_multi_agent_orchestration(test_query, debug=True)

planner = result.get('planner', {})
retrieval = result.get('retrieval') or {}
analyst = result.get('analyst', {})

summary = {
    'run_id': result.get('run_id'),
    'intent': planner.get('intent'),
    'retrieval_needed': planner.get('retrieval_needed'),
    'queries': (planner.get('query_bundle') or {}).get('queries'),
    'retrieval_ok': retrieval.get('ok') if isinstance(retrieval, dict) else None,
    'retrieved_tables': len(retrieval.get('top_tables', [])) if isinstance(retrieval, dict) else 0,
    'analyst_used_financial_evaluator': ((analyst.get('trace') or {}).get('used_financial_evaluator')),
    'answer': analyst.get('answer'),
    'open_issue_codes': [x.get('code') for x in analyst.get('open_issues', []) if isinstance(x, dict)],
    'trace': result.get('orchestrator_trace'),
}

print(json.dumps(summary, indent=2, ensure_ascii=False))

assert planner.get('retrieval_needed') is True, 'Expected retrieval_needed=True for this query.'
assert isinstance(retrieval, dict) and retrieval.get('ok') is True, 'Retrieval failed.'
assert len(retrieval.get('top_tables', [])) >= 1, 'Expected at least one retrieved table.'

if not ((analyst.get('trace') or {}).get('used_financial_evaluator')):
    print('Warning: analyst did not call financial_evaluator in this run.')



[planner_timing_ms] llm_planner_ms=58699 llm_query_expansion_ms=7266 plan_total_ms=65971 step_expand_queries_ms=7266 step_fallback_ms=0 step_llm_plan_ms=58700 step_preextract_ms=0 step_validate_ms=0
[analyst_timing_ms] build_prompt_ms=0 agent_invoke_ms=98931 agent_retry_ms=0 total_ms=98931
[analyst_debug] tool_calls=1 used_financial_evaluator=True
{
  "run_id": "run-1cbc1aa2",
  "intent": "filing_calc",
  "retrieval_needed": true,
  "queries": [
    "What was Apple’s total debt (short-term plus long-term) at year-end 2024?",
    "Total liabilities",
    "Total term debt",
    "Commercial paper",
    "Less: Current portion of term debt"
  ],
  "retrieval_ok": true,
  "retrieved_tables": 1,
  "analyst_used_financial_evaluator": true,
  "answer": "Apple’s total debt (short-term plus long-term) at year-end 2024 was **$96,662 million**.",
  "open_issue_codes": [
    "COMPANY_NAME_DETECTED",
    "TICKER_INFERRED_FROM_COMPANY_NAME"
  ],
  "trace": {
    "total_ms": 182156,
    "planner_timing